##### Gözlemler bağımsız
##### Normallik varsayımı ve varyans homojenliği
##### Küresellik Varsayımı (Mauchly Testi)
##### Küresellik Varsayımı geçerli değilse anova için Greehouse-Geisser testi
##### H0: MÖ = MS1 = MS2 = .... = Mk

##### Sigara içenlerin tedavi öncesi ve tedaviden sonraki süreçteki kan değerleri ortalamları arasında anl. fark var mıdır?

In [1]:
import pandas as pd

In [2]:
data = pd.read_excel("C:/Users/htcso/OneDrive/Masaüstü/pythonProject/datasets/ornek_repeated_anova.xlsx")
data.head()

,Örneklem,TÖ,TS1,TS2,TS3
0,1,70,86,80,87
1,2,78,82,77,80
2,3,61,70,64,72
3,4,70,74,68,88
4,5,89,105,89,101


In [3]:
data_ = pd.melt(data, id_vars=["Örneklem"], value_vars=["TÖ", "TS1", "TS2", "TS3"])
data_.columns = ["Örneklem", "Testler", "Kan Değeri"]
data_.head()

,Örneklem,Testler,Kan Değeri
0,1,TÖ,70
1,2,TÖ,78
2,3,TÖ,61
3,4,TÖ,70
4,5,TÖ,89


##### Normallik Testi

In [5]:
import pingouin as pg

In [6]:
normality = pg.normality(data=data_, dv="Kan Değeri", group="Testler", method="shapiro")
print(normality)

                W      pval  normal
Testler                            
TÖ       0.968747  0.883916    True
TS1      0.912647  0.454037    True
TS2      0.924451  0.537977    True
TS3      0.943039  0.683773    True


##### Küresellik Testi

In [7]:
sphericity = pg.sphericity(data=data_, dv="Kan Değeri", subject="Örneklem", within="Testler")
print(sphericity)

SpherResults(spher=True, W=0.8329041323515878, chi2=0.6805589418687861, dof=5, pval=0.9845576556461296)


In [8]:
# pval=0.9845576556461296 > 0.05 --> Küresellik varsayımı geçerlidir

##### ANOVA

In [9]:
anova = pg.rm_anova(data=data_, dv="Kan Değeri", subject="Örneklem", within="Testler")
print(anova)

    Source  ddof1  ddof2         F     p-unc      ng2       eps
0  Testler      3     15  8.575713  0.001485  0.18215  0.896743


In [10]:
# p-unc 0.001485 < 0.05 --> Gruplar arası anlamlı fark vardır.

##### Post Hoc Testi

In [12]:
ph = pg.pairwise_ttests(data=data_, dv="Kan Değeri", subject="Örneklem", within="Testler", padjust="bonf")
print(ph)

C:\Users\htcso\AppData\Roaming\Python\Python39\site-packages\pingouin\pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


  Contrast    A    B  Paired  Parametric         T  dof alternative     p-unc  \
0  Testler  TS1  TS2    True        True  2.254548  5.0   two-sided  0.073855   
1  Testler  TS1  TS3    True        True -0.909782  5.0   two-sided  0.404659   
2  Testler  TS1   TÖ    True        True  3.849295  5.0   two-sided  0.012010   
3  Testler  TS2  TS3    True        True -2.738328  5.0   two-sided  0.040873   
4  Testler  TS2   TÖ    True        True  1.493703  5.0   two-sided  0.195478   
5  Testler  TS3   TÖ    True        True  4.687983  5.0   two-sided  0.005395   

     p-corr p-adjust    BF10    hedges  
0  0.443127     bonf   1.576  0.478782  
1  1.000000     bonf   0.515 -0.196102  
2  0.072058     bonf   6.012  0.755557  
3  0.245240     bonf    2.42 -0.759886  
4  1.000000     bonf   0.803  0.294251  
5  0.032369     bonf  11.003  1.085784  


In [13]:
# p-corr < 0.05 olan gruplar arasında anlamlı farklılık vardır. (TÖ - TS3)

In [16]:
data_.groupby("Testler")["Kan Değeri"].mean()

Testler
TS1    83.500000
TS2    77.666667
TS3    85.833333
TÖ     74.500000
Name: Kan Değeri, dtype: float64

In [17]:
# Kan değerleri ortalaması TS3'te istenen düzeyde ise tedavinin 3. aşamaya kadar devam etmesinde fayda vardır denebilir.